# Оформление проекта

In [54]:
#import libraries
import pandas as pd
import numpy as np
from scipy import stats
import itertools
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
from matplotlib.colors import PowerNorm, ListedColormap, LogNorm, to_hex
import folium
# define New York area
ny_long_s=-74.25559
ny_long_f=-73.70001
ny_lat_s=40.49612
ny_lat_f=40.91553
ny_range=[[ny_long_s,ny_long_f],[ny_lat_s,ny_lat_f]]

 ##  Кратко о результатах

В рамках проекта были рассмотрены следующие модели:
- SARIMAX
- Линейная регрессия
- Случайный лес без дополнительных признаков
- Случайный лес с дополнительными признаками

Лучший результат показала модель "Случайный лес без дополнительных признаков" (ошибка - 22.46284). Рассмотрим результаты подробнее

In [119]:
results=pd.read_csv('/Users/kirill/YellowTaxi/yTaxi_best_results.csv',
                    parse_dates=[0],
                    infer_datetime_format=True,
                    sep=',')
regions=pd.read_csv('/Users/kirill/YellowTaxi/regions.csv',
                    infer_datetime_format=True,
                    sep=';')
results['region']=results.region.map(int)
results=results.merge(regions).set_index('datehours')
results['err']=np.abs(results.result-results.rides)
results.head()

,region,rides,result,west,east,south,north,err
datehours,,,,,,,,
2016-06-01 00:00:00,1075,26.0,49.865310,-74.022246,-74.011135,40.697437,40.705825,23.865310
2016-06-01 01:00:00,1075,14.0,16.273313,-74.022246,-74.011135,40.697437,40.705825,2.273313
2016-06-01 02:00:00,1075,5.0,16.273313,-74.022246,-74.011135,40.697437,40.705825,11.273313
2016-06-01 03:00:00,1075,2.0,16.273313,-74.022246,-74.011135,40.697437,40.705825,14.273313
2016-06-01 04:00:00,1075,1.0,16.273313,-74.022246,-74.011135,40.697437,40.705825,15.273313


## Визуализация прогоноза для каждой зоны

Построим виджет с прогнозом для каждой зоны

In [135]:

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

def plt_results(reg):
    results[results.region==reg][['rides','result']].plot(figsize=(20,10),title='Region {}. Forecst vs Fact'.format(reg))
    results[results.region==reg][['err']].plot(figsize=(20,7),title='Region {}. Absolute Error'.format(reg))
    plt.show()
    print('MAE= {}'.format(results[results.region==reg][['err']].mean()))
    
interact(plt_results,reg=results.region.unique())

interactive(children=(Dropdown(description='reg', options=(1075, 1076, 1077, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1221, 1222, 1223, 1224, 1225, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1272, 1273, 1274, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1326, 1327, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1376, 1377, 1378, 1380, 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1426, 1431, 1434, 1435, 1436, 1437, 1438, 1439, 1441, 1442, 1480, 1482, 1483, 1530, 1532, 1533, 1580, 1630, 1684, 1733, 1734, 1783, 2068, 2069, 2118, 2119, 2168), value=1075), Output()), _dom_classes=('widget-interact',))

<function __main__.plt_results>

## Отображение реального и прогнозируемого спроса на карте в каждый момент времени

Построим визуализацию реального и прогнозирумого прогноза спроса на такси в каждый момент времени за июнь 2016 года c помощью библиотеки follium

In [137]:

def print_date(day,hour,values):
    to_show=results[(results.index.day==day) &
                    (results.index.hour==hour)]
        
    cm_norm =LogNorm(vmin = to_show[values].min()+1, vmax = to_show[values].max(),clip=True)
    cmap = plt.get_cmap('seismic',7)

    def style_function(x):
                rides = x['properties'][values]
                return {
                    'weight': 1,
                    'opacity': 0.25,
                    'color': '#e0dddd',
                    'fillOpacity': 0.3 if rides>0 else 0,
                    'fillColor': to_hex( cmap(cm_norm(rides))) if rides>0 else '#ffffff',
                    'tooltip':"toto",
                    'title': rides
                }
    def highlight_function(x):
                return {
                    'weight':  1,
                    'fillOpacity': .8
                }


    
    m_ = folium.Map(location=[40.74844, -73.985664],tiles='Stamen Toner',zoom_start=10.5) # center at ESB
    
   
    gdf_yt_f = gpd.GeoDataFrame(to_show[values],geometry = [shg.box(x[4],x[7],x[5],x[6])
                                                  for x in to_show.itertuples()])
    gdf_yt_f.crs = {'init' :'epsg:4326'}

    folium.GeoJson(gdf_yt_f,style_function= style_function,highlight_function=highlight_function).add_to(m_)

    return m_
    
interact(print_date,day=SelectionSlider(description='day',options=results.index.day.unique()),
                    hour=SelectionSlider(description='hour',options=results.index.hour.unique()),
                    values=['rides','result','err'])

interactive(children=(SelectionSlider(description='day', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30), value=1), SelectionSlider(description='hour', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23), value=0), Dropdown(description='values', options=('rides', 'result', 'err'), value='rides'), Output()), _dom_classes=('widget-interact',))

<function __main__.print_date>

## Выводы

Анализ результатов показал, что модель есть куда улучшать:
 * Есть зоны, где модель просто прогнозирует среднее значение на весь месяц
 * Есть зоны со специальными событиями, которые не были учтены в модели
 
Довольно странно, что модель без специальных признаков дает лучший результат. Возможно стоило подойти к построению признаков более аккуратно.

В целом, было познавательно и увлекательно. Спасибо огромное за время потраченное на проверку работы!